In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import scipy.stats
import statsmodels.api as sm
import os
import datetime as dt
import requests
from urllib.parse import urlparse
import json

In [2]:
os.getcwd()

'C:\\Users\\user1\\Desktop\\창원_빅데이터_공모전\\Changwon_BigData_Competition\\Code_preprocessing'

In [3]:
park_df = pd.read_csv('C:\\Users\\user1\\Desktop\\창원_빅데이터_공모전\\Changwon_BigData_Competition\\Raw_data\\Park_Changwon.csv',encoding='cp949')
park_df.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편의시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')

In [4]:
# 분석에 필요한 열만 추출 
park_df = park_df.iloc[:,1:8]
park_df.head()

,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적
0,가호2어린이공원,어린이공원,-,경상남도 진주시 가좌동 1438-7,35.158705,128.106265,855.0
1,초장1지구 역사공원,역사공원,-,경상남도 진주시 초전동 1664,35.211528,128.105637,4849.0
2,물빛여울공원,문화공원,경상남도 진주시 에나로128번길 14,경상남도 진주시 충무공동 291,35.175809,128.142738,13162.0
3,평거17어린이공원,어린이공원,경상남도 진주시 남강로 373번길 3,경상남도 진주시 평거동 794-2번지 일원,35.172144,128.067875,1500.0
4,평거16어린이공원,어린이공원,-,경상남도 진주시 신안동 708-3번지 일원,35.177248,128.070662,1500.0


In [5]:
# 결측치 확인 
for i in park_df.columns:
    print(i,':',len(park_df[park_df[i]=='-']))

공원명 : 0
공원구분 : 0
소재지도로명주소 : 476
소재지지번주소 : 2
위도 : 0
경도 : 0
공원면적 : 0


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
# 결측치 확인 
len(park_df[park_df['소재지도로명주소']=='-'])

476

In [7]:
park_df[park_df['소재지지번주소']=='-']

,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적
387,대치공원,근린공원,경상남도 함안군 칠서면 향촌길 20,-,35.363432,128.488949,14449.0
390,사적공원,근린공원,경상남도 함안군 가야읍 도항1길 46-8,-,35.280446,128.406466,12840.0


In [8]:
# 소재지도로명주소의 결측치가 476개로 너무 많아 소재지지번주소로 통일하고 
# 소재지지번주소의 결측치는 소재지도로명주소로 대체한다 
# 그리고 변수명을 그냥 소재지주소로 변경
park_df['소재지지번주소'][387] = park_df['소재지도로명주소'][387]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
park_df['소재지지번주소'][387] 

'경상남도 함안군 칠서면 향촌길 20'

In [10]:
park_df['소재지지번주소'][390] = park_df['소재지도로명주소'][390]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
park_df['소재지지번주소'][390] 

'경상남도 함안군 가야읍 도항1길 46-8'

In [12]:
# 결측치 확인 
for i in park_df.columns:
    print(i,':',len(park_df[park_df[i]=='-']))

공원명 : 0
공원구분 : 0
소재지도로명주소 : 476
소재지지번주소 : 0
위도 : 0
경도 : 0
공원면적 : 0


In [13]:
# 소재지도로명주소 열 제거 
del park_df['소재지도로명주소']

In [14]:
park_df.rename(columns={'소재지지번주소':'소재지주소'},inplace=True)
park_df.head()

,공원명,공원구분,소재지주소,위도,경도,공원면적
0,가호2어린이공원,어린이공원,경상남도 진주시 가좌동 1438-7,35.158705,128.106265,855.0
1,초장1지구 역사공원,역사공원,경상남도 진주시 초전동 1664,35.211528,128.105637,4849.0
2,물빛여울공원,문화공원,경상남도 진주시 충무공동 291,35.175809,128.142738,13162.0
3,평거17어린이공원,어린이공원,경상남도 진주시 평거동 794-2번지 일원,35.172144,128.067875,1500.0
4,평거16어린이공원,어린이공원,경상남도 진주시 신안동 708-3번지 일원,35.177248,128.070662,1500.0


In [15]:
# 창원시의 공원정보만 추출 
park_Changwon_df = park_df[park_df['소재지주소'].str.contains('창원시')]
park_Changwon_df.head()

,공원명,공원구분,소재지주소,위도,경도,공원면적
608,꿈동산,어린이공원,경상남도 창원시 의창구 봉곡동 5,35.248733,128.655473,691.0
609,무동3호,어린이공원,경상남도 창원시 의창구 북면 무동리 121-4,35.319959,128.581953,10114.0
610,가람공원(어린이공원33),어린이공원,경상남도 창원시 진해구 용원동 1339-4,35.087521,128.813516,3025.0
611,국화,어린이공원,경상남도 창원시 의창구 도계동 341-3,35.259182,128.636181,1599.0
612,개나리,어린이공원,경상남도 창원시 의창구 도계동 358-1,35.260970,128.639965,1710.0


In [16]:
os.getcwd()

'C:\\Users\\user1\\Desktop\\창원_빅데이터_공모전\\Changwon_BigData_Competition\\Code_preprocessing'

In [18]:
park_Changwon_df.to_csv('C:\\Users\\user1\\Desktop\\창원_빅데이터_공모전\\Changwon_BigData_Competition\\Preprocessed_data\\Park_Changwon_df.csv')